## Урок 8

## Дисперсионный анализ. Метод главных компонент. Логистическая регрессия

### Домашнее задание

В поле ответа впишите одно число. Нецелые числа следует округлить до 8 знаков после запятой.

__Задача 1__

Используя данные о хоккеистах (скачать можно [здесь](https://habr.com/ru/post/301340/)), проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие: а) роста, б) BMI.
. В качестве ответа запишите 1, если является, и 0 иначе.


_Подсказка_: не забудьте предобработать данные (например, дублируются ли данные о конкретных игроках?).

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('hockey_players.csv', encoding = 'iso-8859-1')

In [23]:
df['country'].unique()

array(['RUS', 'AUT', 'BLR', 'CAN', 'CZE', 'FIN', 'GER', 'ITA', 'JPN',
       'LAT', 'NOR', 'SUI', 'SVK', 'SWE', 'UKR', 'USA', 'POL', 'SLO',
       'DEN', 'FRA', 'KAZ', 'HUN'], dtype=object)

In [3]:
skand = df.loc[df['country'].isin(['NOR', 'FIN', 'DEN'])]
skand

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
120,2001,FIN,2,kiprusoff marko,D,R,184,89.0,1972-06-06,ehc kloten,28.900753,1972,26.287807
121,2001,FIN,3,nummelin petteri,D,L,178,83.0,1972-11-25,columbus blue jackets,28.429843,1972,26.196187
122,2001,FIN,4,timonen kimmo,D,L,178,88.0,1975-03-18,nashville predators,26.121834,1975,27.774271
123,2001,FIN,40,niemi antti-jussi,D,L,185,87.0,1977-09-22,anaheim mighty ducks,23.605749,1977,25.420015
124,2001,FIN,5,salo sami,D,R,192,99.0,1974-09-02,ottawa senators,26.661191,1974,26.855469
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,2016,NOR,8,trettenes mathias,F,L,179,76.0,1993-11-08,stavanger oilers,22.477755,1993,23.719609
6189,2016,NOR,93,valkvae olsen thomas,F,R,186,88.0,1993-05-18,frisk asker,22.954141,1993,25.436467
6190,2016,NOR,30,haugen lars,G,L,185,83.0,1987-03-19,farjestad karlstad,29.119781,1987,24.251278
6191,2016,NOR,31,volden lars,G,L,190,95.0,1992-07-26,rogle angelholm,23.764545,1992,26.315789


In [4]:
skand['name'].value_counts()

green morten              12
madsen morten             11
nielsen daniel            11
holos jonas               11
bastiansen anders         11
                          ..
smidt michael              1
staal kim                  1
hansen jannik              1
kjaergaard christoffer     1
valkvae olsen thomas       1
Name: name, Length: 430, dtype: int64

In [5]:
Dup_Players = skand[skand[['name']].duplicated()]  
Dup_Players

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
475,2002,FIN,3,nummelin petteri,D,L,178,83.0,1972-11-25,hc lugano,29.429158,1972,26.196187
477,2002,FIN,4,timonen kimmo,D,L,178,88.0,1975-03-18,nashville predators,27.121150,1975,27.774271
483,2002,FIN,14,helminen raimo,F,L,183,90.0,1964-03-11,ilves tampere,38.138261,1964,26.874496
484,2002,FIN,17,kallio tomi,F,L,184,85.0,1977-01-27,atlanta thrashers,25.256674,1977,25.106333
486,2002,FIN,21,kapanen niko,F,L,177,81.0,1978-04-29,utah grizzlies,24.005476,1978,25.854639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6187,2016,NOR,51,rosseli olsen mats,F,L,180,83.0,1991-04-29,frolunda gothenburg,25.007529,1991,25.617284
6188,2016,NOR,8,trettenes mathias,F,L,179,76.0,1993-11-08,stavanger oilers,22.477755,1993,23.719609
6190,2016,NOR,30,haugen lars,G,L,185,83.0,1987-03-19,farjestad karlstad,29.119781,1987,24.251278
6191,2016,NOR,31,volden lars,G,L,190,95.0,1992-07-26,rogle angelholm,23.764545,1992,26.315789


In [6]:
skand_RM_DUP = skand.drop_duplicates(keep='last', subset = 'name')   # Удалим повторы по имени игрока

In [7]:
skand_RM_DUP['name'].value_counts()

niemi antti-jussi      1
korpikoski lauri       1
anttila marko          1
hagman niklas          1
jalasvaara janne       1
                      ..
jokinen olli           1
niemi antti - jussi    1
vaananen ossi          1
timonen kimmo          1
soberg steffen         1
Name: name, Length: 430, dtype: int64

In [10]:
NOR = skand_RM_DUP['height'].loc[skand_RM_DUP['country'] == 'NOR']
FIN = skand_RM_DUP['height'].loc[skand_RM_DUP['country'] == 'FIN']
DEN = skand_RM_DUP['height'].loc[skand_RM_DUP['country'] == 'DEN']
y_NOR = np.array(NOR)
y_FIN  = np.array(FIN )
y_DEN = np.array(DEN)

Требуется проверить гипотезу у равенстве среднего роста по каждой стране(отличие роста незначимо). Для проверки этой гипотезы воспользуемся однофакторным дисперсионным анализом.

In [11]:
n_NOR = y_NOR.shape[0]
n_FIN = y_FIN.shape[0]
n_DEN = y_DEN.shape[0]

n_NOR, n_FIN, n_DEN

(98, 200, 132)

Выборочные средние по каждой группе:

In [12]:
y_NOR_mean = y_NOR.mean()
y_FIN_mean = y_FIN.mean()
y_DEN_mean = y_DEN.mean()

y_NOR_mean, y_FIN_mean, y_DEN_mean

(182.80612244897958, 184.445, 184.3030303030303)

Видно, что средний рост немного отличается. Проверим статистическую значимость этого отличия.

Соберём все значения роста в один массив:

In [13]:
y = np.concatenate([y_NOR, y_FIN, y_DEN])

In [14]:
y_mean = y.mean()
y_mean

184.0279069767442

Найдём значения $SS_b$ и $SS_w$:

In [15]:
SS_b = n_NOR * (y_NOR_mean - y_mean) ** 2 + n_FIN * (y_FIN_mean - y_mean) ** 2 + n_DEN * (y_DEN_mean - y_mean) ** 2

SS_w = ((y_NOR - y_NOR_mean) ** 2).sum() + ((y_FIN - y_FIN_mean) ** 2).sum() + ((y_DEN - y_DEN_mean) ** 2).sum()

SS_b, SS_w

(191.07500186967218, 10922.590114409399)

Запишем оценки дисперсий:

In [16]:
k = 3
n = n_NOR + n_FIN + n_DEN

k1 = k - 1
k2 = n - k

sigma2_b = SS_b / k1
sigma2_w = SS_w / k2

sigma2_b, sigma2_w

(95.53750093483609, 25.579836333511473)

Итак, значение статистики $F$:

In [18]:
F = sigma2_b / sigma2_w
F

3.734875379545526

Зафиксируем уровень значимости  $\alpha = 0.05$ . Для него найдём нужный нам квантиль:

In [19]:
from scipy import stats

In [20]:
alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)
t

3.0168482792459144

Итак, наша критическая область имеет вид:
$$\Omega_\alpha = \left( 3.01, \infty \right)$$

Значение статистики попало в критическую область, поэтому заключаем, что __отличие среднего роста действительно является статистически значимым__

In [22]:
stats.f_oneway(y_NOR, y_FIN, y_DEN)

F_onewayResult(statistic=3.734875379545475, pvalue=0.024659740563714175)

__Ответ__: 1

__б)__ Проверим гипотезу у равенстве BMI по каждой стране(отличие BMI незначимо). Для проверки этой гипотезы воспользуемся однофакторным дисперсионным анализом.

In [24]:
NOR = skand_RM_DUP['bmi'].loc[skand_RM_DUP['country'] == 'NOR']
FIN = skand_RM_DUP['bmi'].loc[skand_RM_DUP['country'] == 'FIN']
DEN = skand_RM_DUP['bmi'].loc[skand_RM_DUP['country'] == 'DEN']
y_NOR = np.array(NOR)
y_FIN  = np.array(FIN )
y_DEN = np.array(DEN)

In [25]:
n_NOR = y_NOR.shape[0]
n_FIN = y_FIN.shape[0]
n_DEN = y_DEN.shape[0]

n_NOR, n_FIN, n_DEN

(98, 200, 132)

In [26]:
y_NOR_mean = y_NOR.mean()
y_FIN_mean = y_FIN.mean()
y_DEN_mean = y_DEN.mean()

y_NOR_mean, y_FIN_mean, y_DEN_mean

(25.84454256708044, 25.948082381435807, 25.437554156321017)

Видно, что средний bmi немного отличается. Проверим статистическую значимость этого отличия.

Соберём все значения bmi в один массив:

In [28]:
y = np.concatenate([y_NOR, y_FIN, y_DEN])

In [29]:
y_mean = y.mean()
y_mean

25.767764643012605

Найдём значения $SS_b$ и $SS_w$:

In [30]:
SS_b = n_NOR * (y_NOR_mean - y_mean) ** 2 + n_FIN * (y_FIN_mean - y_mean) ** 2 + n_DEN * (y_DEN_mean - y_mean) ** 2

SS_w = ((y_NOR - y_NOR_mean) ** 2).sum() + ((y_FIN - y_FIN_mean) ** 2).sum() + ((y_DEN - y_DEN_mean) ** 2).sum()

SS_b, SS_w

(21.473736069964502, 828.6119113476543)

Запишем оценки дисперсий:

In [31]:
k = 3
n = n_NOR + n_FIN + n_DEN

k1 = k - 1
k2 = n - k

sigma2_b = SS_b / k1
sigma2_w = SS_w / k2

sigma2_b, sigma2_w

(10.736868034982251, 1.9405431179101975)

Итак, значение статистики $F$:

In [32]:
F = sigma2_b / sigma2_w
F

5.532919076049678

Зафиксируем уровень значимости  $\alpha = 0.05$ . Для него найдём нужный нам квантиль:

In [33]:
alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)
t

3.0168482792459144

Итак, наша критическая область имеет вид:
$$\Omega_\alpha = \left( 3.01, \infty \right)$$

Значение статистики попало в критическую область, поэтому заключаем, что __bmi действительно является статистически значимым__

In [35]:
stats.f_oneway(y_NOR, y_FIN, y_DEN)

F_onewayResult(statistic=5.532919076049785, pvalue=0.004243187723739065)

__Ответ__: 1